In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

## close range vrs distance range relative to the trollinde radar.

In [2]:
track = pd.read_csv('../saved-files/threshold-15/final-tracks-threshold-15.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00,125.209617,13.245867,34.567482,8.952468
1,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00,123.291260,12.974534,35.577388,20.890860
2,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671,12.780750,34.063904,21.887120
3,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00,115.540119,12.708913,36.584485,22.880821
4,4,5,148.000000,253.920901,3,15,336,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.208000e+06,170920.901252,70.064303,14.426429,2,0 days 00:20:00,110.078668,12.845599,38.092506,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17800,299,4,144.743717,239.312025,4,15,23004,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.204744e+06,156312.025471,70.103689,14.055373,6578,0 days 00:05:00,122.013418,15.240493,41.116443,NaN
17801,300,7,146.461260,240.148842,5,15,23041,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.206461e+06,157148.841602,70.087558,14.073848,6578,0 days 00:10:00,120.179000,14.702386,40.106802,5.970119
17802,298,31,256.000000,174.366873,2,15,22993,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.316000e+06,91366.873305,69.124388,12.259164,6586,0 days 00:00:00,148.655096,15.301029,39.603226,10.944407
17803,299,27,256.815629,176.111734,4,15,23027,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.316816e+06,93111.734429,69.116329,12.301452,6586,0 days 00:05:00,147.098363,15.240493,41.116443,10.943279


### Calculate distance relative to trollinde radar

In [3]:
distance = []
for i in range(len(track)):
    #lat_andenes = 69.141599 # latitude of Andenes
    lat_andenes = 69.2412 # latiude of trollinde radar
    #lon_andenes = 15.684104 # latitude of Andenes
    lon_andenes = 16.0030 # longitude of trollinde radar
    
    distances = tobac.analysis.haversine(track['latitude'][i], track['longitude'][i], lat_andenes, lon_andenes)
    distance.append(distances)

In [4]:
distance[:10]
# np.max(distance)

[125.20961670185432,
 123.29125998307515,
 119.98267085603582,
 115.54011938797738,
 110.07866776406749,
 107.55129320993021,
 147.60176073104904,
 144.40302746758852,
 141.5976392563172,
 138.23183372687132]

In [5]:
%%time
# apped distance to track dataframe
track['distance'] = pd.Series(distance)

CPU times: user 3.01 ms, sys: 5 µs, total: 3.02 ms
Wall time: 3.02 ms


In [6]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00,125.209617,13.245867,34.567482,8.952468
1,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00,123.291260,12.974534,35.577388,20.890860
2,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671,12.780750,34.063904,21.887120
3,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00,115.540119,12.708913,36.584485,22.880821
4,4,5,148.000000,253.920901,3,15,336,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.208000e+06,170920.901252,70.064303,14.426429,2,0 days 00:20:00,110.078668,12.845599,38.092506,NaN


# close domain

In [7]:
# latitude of radar(reference point)
ref_latitude_radar = 69.2412
# longitude of radar
ref_longitude_radar = 16.0030
# first 50km from the trollinde radar on the top
latitude_top_50km = 69.6940 
# first 50km from the trollinde radar on the bottom
latitude_bottom_50km = 68.78878

latitude = ((track['latitude'] >= latitude_bottom_50km) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_top = ((track['latitude'] >= ref_latitude_radar) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_bottom = ((track['latitude'] <= ref_latitude_radar) & (track['latitude'] >= latitude_bottom_50km))

longitude = (track['longitude'] <= ref_longitude_radar)

mask_lat = (latitude  & longitude)

# mask_lat = (from_radar_to_50km_top | from_radar_to_50km_bottom)



close_domain_15 = track[ mask_lat]
close_domain_15

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
18,0,20,255.544546,324.527060,17,15,20,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.315545e+06,241527.059669,69.030445,15.954806,20,0 days 00:00:00,23.538329,13.245867,34.567482,23.879002
19,1,28,258.825739,324.860030,27,15,109,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.318826e+06,241860.030388,69.000377,15.954579,20,0 days 00:05:00,26.876487,12.974534,35.577388,34.826114
20,2,27,262.754401,325.077747,16,15,191,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,242077.747280,68.964541,15.949893,20,0 days 00:10:00,30.868930,12.780750,34.063904,19.899722
21,3,30,266.731982,325.601422,8,15,277,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,242601.421660,68.927977,15.952552,20,0 days 00:15:00,34.924637,12.708913,36.584485,15.920343
22,4,36,270.335063,326.000000,2,15,367,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.330335e+06,243000.000000,68.894920,15.953109,20,0 days 00:20:00,38.597920,12.845599,38.092506,4.975171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17782,299,21,230.066404,210.659054,32,15,23021,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.290066e+06,127659.054235,69.344755,13.190634,6559,0 days 00:20:00,111.284897,15.240493,41.116443,38.802051
17783,300,22,231.102329,212.630474,31,15,23056,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.291102e+06,129630.473694,69.334321,13.238340,6559,0 days 00:25:00,109.327812,14.702386,40.106802,36.812201
17802,298,31,256.000000,174.366873,2,15,22993,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.316000e+06,91366.873305,69.124388,12.259164,6586,0 days 00:00:00,148.655096,15.301029,39.603226,10.944407
17803,299,27,256.815629,176.111734,4,15,23027,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.316816e+06,93111.734429,69.116329,12.301452,6586,0 days 00:05:00,147.098363,15.240493,41.116443,10.943279


In [8]:
%%time
cells_relative_time_to_radar_close = [];

for index, cell_id in enumerate(np.unique(close_domain_15['cell'])):
    if cell_id:
        # get all the closed domain cell id
        cells = close_domain_15[close_domain_15['cell'] == cell_id]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar = all_times[::-1][0]
        relative_time = np.subtract(time_cell_hit_radar, all_times)
        # convert to seconds 
        relative_time = relative_time.seconds // 60
        # print(cell_id, relative_time)
        cells_relative_time_to_radar_close.append(relative_time)

CPU times: user 609 ms, sys: 1.26 ms, total: 611 ms
Wall time: 632 ms


In [9]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_close = pd.Series([item for sublist in cells_relative_time_to_radar_close for item in sublist])
close_domain_15['relative-time-to-radar'] = cells_relative_times_to_radar_close.values

/glade/scratch/noteng/ipykernel_258509/3073702359.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_domain_15['relative-time-to-radar'] = cells_relative_times_to_radar_close.values


In [10]:
close_domain_15 = close_domain_15.reset_index(drop=True)

In [12]:
close_domain_15.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),relative-time-to-radar
0,0,20,255.544546,324.527060,17,15,20,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.315545e+06,241527.059669,69.030445,15.954806,20,0 days 00:00:00,23.538329,13.245867,34.567482,23.879002,20
1,1,28,258.825739,324.860030,27,15,109,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.318826e+06,241860.030388,69.000377,15.954579,20,0 days 00:05:00,26.876487,12.974534,35.577388,34.826114,15
2,2,27,262.754401,325.077747,16,15,191,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,242077.747280,68.964541,15.949893,20,0 days 00:10:00,30.868930,12.780750,34.063904,19.899722,10
3,3,30,266.731982,325.601422,8,15,277,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,242601.421660,68.927977,15.952552,20,0 days 00:15:00,34.924637,12.708913,36.584485,15.920343,5
4,4,36,270.335063,326.000000,2,15,367,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.330335e+06,243000.000000,68.894920,15.953109,20,0 days 00:20:00,38.597920,12.845599,38.092506,4.975171,0


In [13]:
len(close_domain_15)

5008

# distant domain

In [14]:
ref_latitude_point_top = 69.6940 
ref_latitude_point_bottom = 68.78878
# longitude of radar
ref_longitude_radar1 = 16.0030
latitude_top_50km_to_150km = 70.5978 # 100km from first 50km on the top
latitude_bottom_50km_to_150km = 67.8878 # 100km from first 50km on the bottom

latitude1 = ((track['latitude'] >= latitude_bottom_50km_to_150km) & (track['latitude'] <= latitude_top_50km_to_150km))
longitude1 = (track['longitude'] <= ref_longitude_radar1)

mask_lat = (latitude1 & longitude1)


distant_domain_15 = track[ mask_lat ]
distant_domain_15

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00,125.209617,13.245867,34.567482,8.952468
1,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00,123.291260,12.974534,35.577388,20.890860
2,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671,12.780750,34.063904,21.887120
3,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00,115.540119,12.708913,36.584485,22.880821
4,4,5,148.000000,253.920901,3,15,336,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.208000e+06,170920.901252,70.064303,14.426429,2,0 days 00:20:00,110.078668,12.845599,38.092506,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17800,299,4,144.743717,239.312025,4,15,23004,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.204744e+06,156312.025471,70.103689,14.055373,6578,0 days 00:05:00,122.013418,15.240493,41.116443,NaN
17801,300,7,146.461260,240.148842,5,15,23041,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.206461e+06,157148.841602,70.087558,14.073848,6578,0 days 00:10:00,120.179000,14.702386,40.106802,5.970119
17802,298,31,256.000000,174.366873,2,15,22993,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.316000e+06,91366.873305,69.124388,12.259164,6586,0 days 00:00:00,148.655096,15.301029,39.603226,10.944407
17803,299,27,256.815629,176.111734,4,15,23027,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.316816e+06,93111.734429,69.116329,12.301452,6586,0 days 00:05:00,147.098363,15.240493,41.116443,10.943279


In [15]:
%%time
cells_relative_time_to_radar_distant = [];

for index, cell_idx in enumerate(np.unique(distant_domain_15['cell'])):
    if cell_idx:
        # get all the closed domain cell id
        cells = distant_domain_15[distant_domain_15['cell'] == cell_idx]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar1 = all_times[::-1][0]
        relative_time1 = np.subtract(time_cell_hit_radar1, all_times)
        # convert to seconds 
        relative_time1 = relative_time1.seconds // 60
        # print(cell_idx, relative_time1)
        cells_relative_time_to_radar_distant.append(relative_time1)

CPU times: user 1.32 s, sys: 2.07 ms, total: 1.32 s
Wall time: 1.36 s


In [16]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_distant = pd.Series([item for sublist in cells_relative_time_to_radar_distant for item in sublist])
distant_domain_15['relative-time-to-radar'] = cells_relative_times_to_radar_distant.values

/glade/scratch/noteng/ipykernel_258509/2074390270.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distant_domain_15['relative-time-to-radar'] = cells_relative_times_to_radar_distant.values


In [17]:
distant_domain_15 = distant_domain_15.reset_index(drop=True)


In [18]:
distant_domain_15.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),relative-time-to-radar
0,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00,125.209617,13.245867,34.567482,8.952468,25
1,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00,123.291260,12.974534,35.577388,20.890860,20
2,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671,12.780750,34.063904,21.887120,15
3,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00,115.540119,12.708913,36.584485,22.880821,10
4,4,5,148.000000,253.920901,3,15,336,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.208000e+06,170920.901252,70.064303,14.426429,2,0 days 00:20:00,110.078668,12.845599,38.092506,NaN,5


In [19]:
len(distant_domain_15)

12947

In [20]:
len(close_domain_15)

5008

In [21]:
len(track)

17805

In [22]:
len(close_domain_15) + len(distant_domain_15)

17955

In [23]:
distant_domain_15['latitude'].max(), distant_domain_15['latitude'].min(),close_domain_15['latitude'].max(), close_domain_15['latitude'].min()

(70.59680938720703, 67.88795469378817, 69.69388640969048, 68.78919651359978)

In [25]:
distant_domain_15['longitude'].max()

16.002740385421976

In [26]:
close_domain_15.to_csv('../saved-files/threshold-15/close-domain-threshold-15.csv', index=False)
distant_domain_15.to_csv('../saved-files/threshold-15/distant-domain-threshold-15.csv', index=False)

# END

In [42]:
# max_dBZ_10 = []
# for indexes, time in enumerate(np.unique(close_domain_10['relative-time-to-radar'])):
#     # Sorting based on same relative times
#     sort_relative_times_threshold_10 = close_domain_10[close_domain_10['relative-time-to-radar'] == time]
#     # Find each relative time associated with a particular reflectivity value
    
#     max_dbZ_threshold_10 = np.mean(sort_relative_times_threshold_10['max dBZ'])
#     max_dBZ_10.append(max_dbZ_threshold_10)